In [ ]:
!pip install overpy

In [ ]:
from math import radians, sin, cos, acos, pi
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import overpy
import plotly.express as px
import pandas as pd
import numpy as np
import scipy

api = overpy.Overpass()

def getdist(a, b):

    mlat = radians(float(a[0]))
    mlon = radians(float(a[1]))
    plat = radians(float(b[0]))
    plon = radians(float(b[1]))
    dist = 6371.01 * acos(sin(mlat)*sin(plat) + cos(mlat)*cos(plat)*cos(mlon - plon))

    return dist

def getnodes(lat=53.288417917647095, lon=-6.140863082085563, radius=1000):
    query = """(node["shop"](around:{rad},{lat},{lon});
            node["office"](around:{rad},{lat},{lon});
            node["healthcare"](around:{rad},{lat},{lon});
            node["building"="commercial"](around:{rad},{lat},{lon});
            node["amenities"="restaurant"](around:{rad},{lat},{lon});
            node["amenities"="cafe"](around:{rad},{lat},{lon});
        );out;
        """.format(lat=lat, lon=lon, rad=radius)
    result = api.query(query)
    nodes = [[float(x.lat), float(x.lon)] for x in result.nodes]
    return nodes


In [ ]:
def nncalc(nodes, radius):
    dist = {}
    dist_val = 0

    for i, node in enumerate(nodes[:-1]):
        dist[i] = 1e9
        for j in range(i+1, len(nodes)):
            dist[i] = min(dist[i], getdist(node, nodes[j]))

        dist_val += dist[i]


    n = len(nodes)
    dist_val /= n

    A = (radius / 1000) ** 2 * pi
    d_expected = 1 / ((n / A) ** 0.5)

    R = dist_val / d_expected

    return R

def run(centroid, radius=1000):
    nodes = getnodes(lat=centroid[0], lon=centroid[1], radius=radius)

    return nncalc(nodes, radius)


def getline(x, y, deg=3):
    coefficients = np.polyfit(x, y, deg)
    predicted_y = np.polyval(coefficients, x)
    error = np.mean((y - predicted_y) ** 2)
    best_fit_line = np.polyval(coefficients, x)
    return best_fit_line, error

def getcentroid(coordinates):

    total_lat = 0
    total_lon = 0
    num_points = len(coordinates)

    for lon, lat in coordinates:
        total_lon += lon
        total_lat += lat

    centroid_lon = total_lon / num_points
    centroid_lat = total_lat / num_points

    return (centroid_lon, centroid_lat)

def update(curlat, curlon, steps, prevloss, radius=1000, discrete=False, maxsteps=100):

    if steps >= maxsteps:
        return (curlat, curlon)

    radial_nodes = getnodes(curlat, curlon, radius)
    centroid = getcentroid(radial_nodes)
    if discrete:
        mn_dst_node = radial_nodes[0]
        mn_dst = getdist(mn_dst_node, centroid)

        for node in radial_nodes:
            cur_dist = getdist(node, centroid)
            if cur_dist < mn_dst:
                mn_dst_node = node
                mn_dst = cur_dist
    else:
        mn_dst_node = centroid

    loss = nncalc(radial_nodes, radius)
    print("loss:", loss)

    if loss == 0 or (loss == prevloss and discrete == True):
        return (curlat, curlon)

    return update(mn_dst_node[0], mn_dst_node[1], steps+1, loss, radius, discrete, maxsteps)


In [ ]:
centroid = update(53.30236, -6.127176, 0, 1000, 2000, True, 10)

In [ ]:
print(centroid)

In [ ]:
print(run(centroid, 2000))

In [ ]:
x = [i for i in range(200, 2600, 100)]
y = [run(centroid, i) for i in x]

plt.scatter(x, y, color="blue", label=f'Points')

# Plot the best fit line
best_fit_line, error = getline(x, y)
plt.plot(x, best_fit_line, color="blue", label=f'Best Fit Line')

# Add labels and title
plt.xlabel('Radius (m)')
plt.ylabel('NN Correlation')
plt.title(f'Best Fit Line for radius-NN correlation')
# Output error and min/max slopes
print(f"Mean Squared Error: {error}")



In [ ]:
# Experimentally determined

radius = 2000

In [ ]:
nodes = getnodes(lat=centroid[0], lon=centroid[1], radius=radius)
rcx, rcy = [], []

for coord in nodes:

    x = - cos(coord[0]) * sin(coord[1])
    z = - sin(coord[0])

    rcx.append(x)
    rcy.append(z)

plt.scatter(rcx, rcy, color="blue", label='Commercial Building')
plt.scatter([0.145], [-0.115], color="yellow", label='Dun Laoghaire Marine Road')
plt.scatter([- cos(centroid[0]) * sin(centroid[1])], [- sin(centroid[0])], color="green", label='Centroid')




# Add labels and title
plt.title('Coordinates Mapped to Cartesian Plane')
plt.legend()
plt.show()

In [ ]:
fig = px.scatter_mapbox(getnodes(radius=radius),
                        lat=0,
                        lon=1,
                        zoom=8,
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
from math import *

# https://stackoverflow.com/questions/58482966/itm-irish-transverse-coordinate-conversion-to-gps-for-google-maps-python3

############################################################################

# Meridian Arc

############################################################################

def arcmer(a,equad,lat1,lat2):

    b=a*sqrt(1-equad)

    n=(a-b)/(a+b)

    a0=1.+((n**2)/4.)+((n**4)/64.)

    a2=(3./2.)*(n-((n**3)/8.))

    a4=(15./16.)*((n**2)-((n**4)/4.))

    a6=(35./48.)*(n**3)



    s1=a/(1+n)*(a0*lat1-a2*sin(2.*lat1)+a4*sin(4.*lat1)-a6*sin(6.*lat1))

    s2=a/(1+n)*(a0*lat2-a2*sin(2.*lat2)+a4*sin(4.*lat2)-a6*sin(6.*lat2))

    return s2-s1

###############################################################################
#
# Transverse Mercator Inverse Projection
#
###############################################################################
def xy2geo(m,p,a,equad,lat0,lon0):

    lat0=radians(lat0)
    lon0=radians(lon0)

    sigma1=p

    fil=lat0+sigma1/(a*(1-equad))

    deltafi=1

    while deltafi > 0.0000000001:

        sigma2=arcmer(a,equad,lat0,fil)

        RO=a*(1-equad)/((1-equad*(sin(fil)**2))**(3./2.))

        deltafi=(sigma1-sigma2)/RO

        fil=fil+deltafi


    N=a/sqrt(1-equad*(sin(fil))**2)

    RO=a*(1-equad)/((1-equad*(sin(fil)**2))**(3./2.))

    t=tan(fil)

    psi=N/RO

    lat=fil-(t/RO)*((m**2)/(2.*N))+(t/RO)*((m**4)/(24.*(N**3)))*(-4.*(psi**2)-9.*psi*(1.-t**2)+12.*(t**2))-(t/RO)*(m**6/(720.*(N**5)))*(8.*(psi**4)*(11.-24.*(t**2))-12.*(psi**3)*(21.-71.*(t**2))+15.*(psi**2)*(15.-98.*(t**2)+15.*(t**4))+180.*psi*(5.*(t**2)-3.*(t**4))-360.*(t**4))+(t/RO)*((m**8)/(40320.*(N**7)))*(1385.+3633.*(t**2)+4095.*(t**4)+1575.*(t**6))

    lon=(m/(N))-((m**3)/(6.*(N**3)))*(psi+2.*(t**2))+((m**5)/(120.*(N**5)))*(-4.*(psi**3)*(1.-6.*(t**2))+(psi**2)*(9.-68.*(t**2))+72.*psi*(t**2)+24.*(t**4))-((m**7)/(5040.*(N**7)))*(61.+662.*(t**2)+1320.*(t**4)+720.*(t**6))

    lon=lon0+lon/cos(fil)

    lat=degrees(lat)
    lon=degrees(lon)

    return lat,lon


#############################################################################

# Irish Transverse Mercator - Inverse

#############################################################################

def itm2geo(x,y):

    # GRS-80

    a = 6378137.

    equad =0.00669437999

    # Natural Origin

    lat0=53.5

    lon0=-8.

    k0=0.999820

    p = (y - 750000.) /k0

    m = (x - 600000.) /k0

    lat,lon = xy2geo(m,p,a,equad,lat0,lon0)

    return lat,lon


In [ ]:
import requests

url = "https://api.valoff.ie/api/Property/GetProperties"
params = {
    "Fields": "AreaPerFloor,RateableValuation,ITM,Eircode",
    "LocalAuthority": "DUN LAOGHAIRE RATHDOWN CO CO",
    "CategorySelected": "OFFICE,FUEL/DEPOT,LEISURE,INDUSTRIAL USES,HEALTH,HOSPITALITY,MINERALS,MISCELLANEOUS,RETAIL (SHOPS),UTILITY,RETAIL (WAREHOUSE),NO CATEGORY SELECTED,CENTRAL VALUATION LIST",
    "Format": "json",
    "Download": "false"
}

response = requests.get(url, params=params)

data = response.json()

print(data)

In [ ]:
filtered = []

for loc in data:
    if 'Eircode' in loc and loc['Eircode'] is not None and len(loc['Eircode']) > 3:
        lat, lon = itm2geo(loc["Xitm"], loc["Yitm"])

        if getdist([lat, lon], centroid) > radius / 1000:
            continue

        if loc["Valuation"] > 1 * 10 ** 6:
            loc["Valuation"] = 1 * 10 ** 6
        area = 0.0

        for dict_obj in loc["ValuationReport"]:
            area += dict_obj["Area"]
        print(area, getdist([lat, lon], centroid))
        if area > 0 and loc["Valuation"] / area < 1000:
            filtered.append([lat, lon, len(loc["ValuationReport"]), loc["Valuation"] / area])

In [ ]:
print(len(filtered))
print(filtered[0])

In [ ]:
color_scale = [(0, 'blue'), (1,'red')]


fig = px.scatter_mapbox(filtered,
                        lat=0,
                        lon=1,
                        hover_name=3,
                        color=3,
                        color_continuous_scale=color_scale,
                        size=2,
                        zoom=8,
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
def func(x, a, b, c):
    return a * np.array(np.exp(-b * x)) + c

def curvefit(x, y):
    popt, pcov = curve_fit(func, x, y, maxfev=10000)
    return popt

In [ ]:
mp = {}

for a in filtered:
    cur_dist = round(int(getdist([a[0], a[1]], [53.2892096965318, -6.139133507514453]) * 1000), -2)
    if cur_dist not in mp: mp[cur_dist] = 0
    mp[cur_dist] = max(mp[cur_dist], a[2])

x = list(mp.keys())
y = list(mp.values())

plt.scatter(x, y, color="blue", label=f'Points')

# Plot the best fit line
best_fit_line, error = getline(x, y, deg=1)
plt.plot(x, best_fit_line, color="black", label=f'Best Fit Line')

# Add labels and title
plt.xlabel('Distance (m)')
plt.ylabel('Height Correlation')
plt.title(f'Best Fit Line for Central Distance-Height Correlation')
# Output error and min/max slopes
print(f"Mean Squared Error: {error}")


In [ ]:
mp = {}

for a in filtered:
    cur_dist = round(int(getdist([a[0], a[1]], [53.2892096965318, -6.139133507514453]) * 1000), -2)
    if cur_dist not in mp: mp[cur_dist] = 0
    mp[cur_dist] = max(mp[cur_dist], a[3])

x = list(mp.keys())
y = list(mp.values())

plt.scatter(x, y, color="blue", label=f'Points')

# Plot the best fit line
best_fit_line, error = getline(x, y, deg=1)
plt.plot(x, best_fit_line, color="black", label=f'Best Fit Line')

# Add labels and title
plt.xlabel('Distance (m)')
plt.ylabel('Value Correlation')
plt.title(f'Best Fit Line for Central Distance-Value Correlation')
# Output error and min/max slopes
print(f"Mean Squared Error: {error}")
